<a href="https://colab.research.google.com/github/flydragon2018/Analytics_Vidhya/blob/master/catboost_bookrating_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
ls drive/"My Drive"

 360CC_config.yaml                          mymlabel.tar
 5ed0795572319.rar                          mytest.csv
 71_OCR_2.zip                               ocr/
 721099_1254334_bundle_archive.zip          ocr2/
 75.zip                                     OWN_config.yaml
 83_face.zip                                refuse_classification.zip
 85book_Rating.zip                          resnet34_sub.csv
 b7_submission.json                         result.txt
 book.csv                                   round2label.zip
 chineseocr@                                submission.csv
 chinese_ocr_test0710.py                    surface_defect/
 ColabNotebooks/                            test.csv
'Colab Notebooks'/                          test_own.csv
 colab_train.py                             tf_refuse.zip
'Copy of NEU surface defect database.rar'   train.csv
 data/                                      train_myimg627.py
 data.zip                                   train_myimg.py
 efficientnet3_sub.cs

In [7]:
!unzip -q   /content/drive/"My Drive"/85book_Rating.zip  

In [8]:
ls

book.csv  drive/  sample_data/  test.csv  train.csv  user.csv


In [28]:

import pandas as pd


traindata = pd.read_csv('train.csv')
traindata.head()

,User-ID,ISBN,Book-Rating
0,8,0002005018,5
1,8,0374157065,0
2,8,0393045218,0
3,8,0399135782,0
4,8,0425176428,0


In [29]:
books = pd.read_csv("book.csv",encoding='latin-1')
books.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [32]:
books.loc[books["Year-Of-Publication"]=='Amit Chaudhuri']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher


In [31]:
books.loc[books["ISBN"]=='0330482750', "Year-Of-Publication"]=2001
books.loc[books["ISBN"]=='0330482750', "Publisher"]="Pan MacMillan"
books.loc[books["ISBN"]=='0330482750', "Book-Author"]="Amit Chaudhuri"
books.loc[books["ISBN"]=='0330482750', "Book-Title"]=" A Strange and Sublime Address, Afternoon Raag,Freedom Song"

In [33]:
userdf = pd.read_csv("user.csv",encoding='latin-1')
userdf.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [34]:
userdf["Age"]=userdf["Age"].fillna(int(-1))

In [35]:
traindata = traindata.merge(books,on='ISBN', how='left',suffixes=('_left', '_right'))
traindata.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,8,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
1,8,0374157065,0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
2,8,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company
3,8,0399135782,0,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group
4,8,0425176428,0,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group


In [36]:
traindata = traindata.merge(userdf,on='User-ID', how='left')
traindata.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age
0,8,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"timmins, ontario, canada",-1.0
1,8,0374157065,0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"timmins, ontario, canada",-1.0
2,8,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,"timmins, ontario, canada",-1.0
3,8,0399135782,0,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,"timmins, ontario, canada",-1.0
4,8,0425176428,0,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,"timmins, ontario, canada",-1.0


In [37]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737864 entries, 0 to 737863
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              737864 non-null  int64  
 1   ISBN                 737864 non-null  object 
 2   Book-Rating          737864 non-null  int64  
 3   Book-Title           737864 non-null  object 
 4   Book-Author          737863 non-null  object 
 5   Year-Of-Publication  737864 non-null  object 
 6   Publisher            737863 non-null  object 
 7   Location             737864 non-null  object 
 8   Age                  737864 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 56.3+ MB


In [38]:
traindata.to_csv("mybook_train.csv",index=None)

In [39]:
testdata = pd.read_csv('test.csv')
testdata.head()

,id,User-ID,ISBN
0,0,8,0060973129
1,1,8,0671870432
2,2,8,074322678X
3,3,8,0771074670
4,4,14,0689821166


In [40]:
testdata = testdata.merge(books,on='ISBN', how='left',suffixes=('_left', '_right'))
testdata.head()

,id,User-ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0,8,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
1,1,8,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks
2,2,8,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner
3,3,8,0771074670,Nights Below Station Street,David Adams Richards,1988,Emblem Editions
4,4,14,0689821166,Flood : Mississippi 1927,Kathleen Duey,1998,Aladdin


In [41]:
testdata = testdata.merge(userdf,on='User-ID', how='left')
testdata.head()

,id,User-ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age
0,0,8,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"timmins, ontario, canada",-1.0
1,1,8,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,"timmins, ontario, canada",-1.0
2,2,8,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,"timmins, ontario, canada",-1.0
3,3,8,0771074670,Nights Below Station Street,David Adams Richards,1988,Emblem Editions,"timmins, ontario, canada",-1.0
4,4,14,0689821166,Flood : Mississippi 1927,Kathleen Duey,1998,Aladdin,"mediapolis, iowa, usa",-1.0


In [42]:
testdata.to_csv("mybook_test.csv",index=None)

In [43]:
list(traindata.columns.values)

['User-ID',
 'ISBN',
 'Book-Rating',
 'Book-Title',
 'Book-Author',
 'Year-Of-Publication',
 'Publisher',
 'Location',
 'Age']

In [44]:
list(testdata)

['id',
 'User-ID',
 'ISBN',
 'Book-Title',
 'Book-Author',
 'Year-Of-Publication',
 'Publisher',
 'Location',
 'Age']

In [45]:
traindf=pd.read_csv("mybook_train.csv")
traindf.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age
0,8,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"timmins, ontario, canada",-1.0
1,8,0374157065,0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"timmins, ontario, canada",-1.0
2,8,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,"timmins, ontario, canada",-1.0
3,8,0399135782,0,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,"timmins, ontario, canada",-1.0
4,8,0425176428,0,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,"timmins, ontario, canada",-1.0


In [46]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737864 entries, 0 to 737863
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              737864 non-null  int64  
 1   ISBN                 737864 non-null  object 
 2   Book-Rating          737864 non-null  int64  
 3   Book-Title           737864 non-null  object 
 4   Book-Author          737863 non-null  object 
 5   Year-Of-Publication  737864 non-null  int64  
 6   Publisher            737863 non-null  object 
 7   Location             737864 non-null  object 
 8   Age                  737864 non-null  float64
dtypes: float64(1), int64(3), object(5)
memory usage: 50.7+ MB


In [49]:
testdf=pd.read_csv("mybook_test.csv")
testdf.head()

,id,User-ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age
0,0,8,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"timmins, ontario, canada",-1.0
1,1,8,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,"timmins, ontario, canada",-1.0
2,2,8,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,"timmins, ontario, canada",-1.0
3,3,8,0771074670,Nights Below Station Street,David Adams Richards,1988,Emblem Editions,"timmins, ontario, canada",-1.0
4,4,14,0689821166,Flood : Mississippi 1927,Kathleen Duey,1998,Aladdin,"mediapolis, iowa, usa",-1.0


In [50]:
test=testdf.drop("id",axis=1)
test.head()

,User-ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age
0,8,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"timmins, ontario, canada",-1.0
1,8,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,"timmins, ontario, canada",-1.0
2,8,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,"timmins, ontario, canada",-1.0
3,8,0771074670,Nights Below Station Street,David Adams Richards,1988,Emblem Editions,"timmins, ontario, canada",-1.0
4,14,0689821166,Flood : Mississippi 1927,Kathleen Duey,1998,Aladdin,"mediapolis, iowa, usa",-1.0


In [52]:
!pip install catboost

     |████████████████████████████████| 65.9MB 65kB/s 


In [54]:
catX = traindf.drop(['Book-Rating'], axis = 1)

In [55]:
catX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737864 entries, 0 to 737863
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              737864 non-null  int64  
 1   ISBN                 737864 non-null  object 
 2   Book-Title           737864 non-null  object 
 3   Book-Author          737863 non-null  object 
 4   Year-Of-Publication  737864 non-null  int64  
 5   Publisher            737863 non-null  object 
 6   Location             737864 non-null  object 
 7   Age                  737864 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 45.0+ MB


In [56]:
caty=traindf['Book-Rating'] 

In [57]:
catX["ISBN"]=catX["ISBN"].astype(str)
catX["Book-Title"]=catX["Book-Title"].astype(str)
catX["Book-Author"]=catX["Book-Author"].astype(str)
catX["Publisher"]=catX["Publisher"].astype(str)
catX["Location"]=catX["Location"].astype(str)
catX["Year-Of-Publication"]=catX["Year-Of-Publication"].astype(int)   

In [60]:
testX=test

In [62]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206235 entries, 0 to 206234
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              206235 non-null  int64  
 1   ISBN                 206235 non-null  object 
 2   Book-Title           206235 non-null  object 
 3   Book-Author          206235 non-null  object 
 4   Year-Of-Publication  206235 non-null  int64  
 5   Publisher            206234 non-null  object 
 6   Location             206235 non-null  object 
 7   Age                  206235 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 12.6+ MB


In [63]:
testX["ISBN"]=testX["ISBN"].astype(str)
testX["Book-Title"]=testX["Book-Title"].astype(str)
testX["Book-Author"]=testX["Book-Author"].astype(str)
testX["Publisher"]=testX["Publisher"].astype(str)
testX["Location"]=testX["Location"].astype(str)
testX["Year-Of-Publication"]=testX["Year-Of-Publication"].astype(int)  

In [64]:
from catboost import CatBoostClassifier


# Initialize CatBoostClassifier
model = CatBoostClassifier()
cat_features =['ISBN','Book-Title','Book-Author', 'Publisher', 'Location']


# Fit model
model.fit(catX, caty, cat_features)
# Get predicted classes
preds_class = model.predict(testX)
print(preds_class)

p=preds_class
result=[]
for k in range(len(p)):
    #print(k,p[k])
    result.append([k,p[k].numpy()])
submit=pd.DataFrame(result)
submit.to_csv("cat_np_submission.csv",index=None,header=None)


Learning rate set to 0.110813
0:	learn: 1.9488214	total: 23.9s	remaining: 6h 38m 44s
1:	learn: 1.7643316	total: 47.2s	remaining: 6h 32m 53s
2:	learn: 1.6453567	total: 1m 10s	remaining: 6h 27m 45s
3:	learn: 1.5600784	total: 1m 33s	remaining: 6h 29m 2s
4:	learn: 1.4955967	total: 1m 55s	remaining: 6h 24m 9s
5:	learn: 1.4447590	total: 2m 18s	remaining: 6h 21m 22s
6:	learn: 1.4035243	total: 2m 41s	remaining: 6h 20m 59s
7:	learn: 1.3696466	total: 3m 3s	remaining: 6h 18m 44s
8:	learn: 1.3410057	total: 3m 33s	remaining: 6h 31m 11s
9:	learn: 1.3174180	total: 3m 58s	remaining: 6h 33m 49s
10:	learn: 1.2968471	total: 4m 29s	remaining: 6h 44m 1s
11:	learn: 1.2801384	total: 4m 56s	remaining: 6h 47m 25s
12:	learn: 1.2657919	total: 5m 24s	remaining: 6h 50m 8s
13:	learn: 1.2536680	total: 5m 51s	remaining: 6h 52m 21s
14:	learn: 1.2430445	total: 6m 21s	remaining: 6h 57m 52s
15:	learn: 1.2334854	total: 6m 52s	remaining: 7h 2m 52s
16:	learn: 1.2256563	total: 7m 20s	remaining: 7h 4m 19s
17:	learn: 1.2179502

In [66]:
!cp *.csv /content/drive/"My Drive"/bookrating   

In [69]:
p=preds_class
result=[]
for k in range(len(p)):
    #print(k,p[k])
    result.append([k,int(p[k])])
submit=pd.DataFrame(result)
submit.to_csv("cat_np_submission.csv",index=None,header=None)
